# Agenda:

* Data read and feature transformations
* Keras Model
* Keras Training loop

## Import Necessary Libraries

In [4]:
!pip install keras

In [10]:
import keras
print(keras.__version__)

2.13.1


In [12]:
!pip install tensorflow

  Obtaining dependency information for typing-extensions<4.6.0,>=3.6.6 from https://files.pythonhosted.org/packages/31/25/5abcd82372d3d4a3932e1fa8c3dbf9efac10cc7c0d16e78467460571b404/typing_extensions-4.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<2.0 from https://files.pythonhosted.org/packages/33/cf/8435d5a7159e2a9c83a95896ed596f68cf798005fe107cc655b5c5c14704/urllib3-1.26.20-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.0 MB/s eta 0:00:00
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 3.2 MB/s eta 0:00:0000:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.34 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.23.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
azure-storage-blob 12.23.1 requires typin

In [14]:
import tensorflow 
print(tensorflow.__version__)

2.13.0


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

### Part 1: Import the Housing data and do feature transformations

In [16]:
df = pd.read_csv("house_price_full.csv")
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


#### Feature transformations

In [44]:
X = df.copy()

# Remove target from x and store in Y
Y = X.pop('price')

# Perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Perform log transformations of target variable
Y = np.log(Y)

In [45]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [46]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

### Part 3: Create Model Using `keras`

In [47]:
from tensorflow import keras

In [48]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2)                 6         
                                                                 
 dense_7 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [50]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [51]:
model.fit(X, Y.values, epochs=10, batch_size=32)

Epoch 1/10
16/16 [==============================] - 0s 472us/step - loss: 111.4208
Epoch 2/10
16/16 [==============================] - 0s 351us/step - loss: 32.3976
Epoch 3/10
16/16 [==============================] - 0s 355us/step - loss: 5.9406
Epoch 4/10
16/16 [==============================] - 0s 367us/step - loss: 1.0412
Epoch 5/10
16/16 [==============================] - 0s 327us/step - loss: 0.3012
Epoch 6/10
16/16 [==============================] - 0s 396us/step - loss: 0.1923
Epoch 7/10
16/16 [==============================] - 0s 322us/step - loss: 0.1739
Epoch 8/10
16/16 [==============================] - 0s 314us/step - loss: 0.1694
Epoch 9/10
16/16 [==============================] - 0s 342us/step - loss: 0.1669
Epoch 10/10
16/16 [==============================] - 0s 359us/step - loss: 0.1649


In [53]:
model.predict(X)[:,0]

16/16 [==============================] - 0s 432us/step


array([12.869589 , 13.561279 , 13.026676 , 13.042841 , 13.160242 ,
       12.560898 , 12.707465 , 13.320253 , 13.1315565, 13.047585 ,
       12.972546 , 13.354694 , 13.11249  , 12.791172 , 13.458633 ,
       12.826574 , 12.935387 , 13.38287  , 12.878512 , 12.820242 ,
       13.094538 , 13.026903 , 12.982771 , 13.3236885, 13.070541 ,
       13.062893 , 13.118305 , 13.220631 , 13.443358 , 13.161919 ,
       13.0591545, 13.345219 , 13.130011 , 13.338741 , 13.447006 ,
       13.25523  , 12.533363 , 13.007584 , 13.033665 , 13.124039 ,
       12.938112 , 13.035835 , 12.550661 , 12.987819 , 12.666496 ,
       12.813864 , 13.0359745, 13.026676 , 12.848366 , 13.409676 ,
       13.320253 , 13.211031 , 13.180241 , 12.99531  , 13.321044 ,
       13.15531  , 13.218433 , 13.562414 , 12.863583 , 12.782766 ,
       13.409676 , 13.175344 , 13.3236885, 13.151491 , 13.354694 ,
       12.567663 , 12.956884 , 12.764532 , 12.993849 , 13.086311 ,
       13.287785 , 13.202126 , 13.237856 , 13.019594 , 12.8665